# Model study

In [1]:
%load_ext autoreload
%autoreload 2
%run _model/__common.py

from modulus.datapipes.climate.era5_hdf5_new import ERA5HDF5Datapipe
datapipe = ERA5HDF5Datapipe(
    base_path="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0",
    data_folder="train",
    shuffle=False,
)

data = next(iter(datapipe))
input = data[0]["input"]
input.shape

/iopsstor/scratch/cscs/stefschu/DSM500/github/modulus-a5275d8/modulus/distributed/manager.py:346: UserWarning: Could not initialize using ENV, SLURM or OPENMPI methods. Assuming this is a single process job
  warn(


__common.py loaded


torch.Size([1, 1, 31, 721, 1440])

## Model

In [2]:
from modulus.models.graphcast import GraphCastNet

model = GraphCastNet(
    mesh_level=6,
    multimesh=True,
    input_res=(721, 1440),
    input_dim_grid_nodes=31,
    input_dim_mesh_nodes=3,
    input_dim_edges=4,
    output_dim_grid_nodes=21,
    processor_type="MessagePassing",
    khop_neighbors=32,
    num_attention_heads=4,
    processor_layers=16,
    hidden_dim=512,
    norm_type="TELayerNorm",
    do_concat_trick=True,
    use_cugraphops_encoder=False,
    use_cugraphops_processor=False,
    use_cugraphops_decoder=False,
    recompute_activation=True,
)

input_gpu = input.to(dtype=torch.bfloat16).to("cuda:0")
model_gpu = model.to(torch.bfloat16).to("cuda:0")
output_gpu = model_gpu(input_gpu.squeeze(0))

print(input_gpu.shape, output_gpu.shape)

del input_gpu, output_gpu, model_gpu
torch.cuda.empty_cache()

GraphCastNet initialization ...
GraphCastNet initialization done
Started forward pass ...
A torch.Size([1, 31, 721, 1440]) torch.Size([31, 721, 1440]) -> torch.Size([1038240, 31])
Done with forward pass ...


(torch.Size([1, 1, 31, 721, 1440]), torch.Size([1, 21, 721, 1440]))

In [3]:
del input_gpu, output_gpu, model_gpu
torch.cuda.empty_cache()

In [17]:
input.shape
input.squeeze(0).squeeze(0).shape
input.squeeze(0).squeeze(0).view(31, -1).permute(1, 0).shape

torch.Size([1038240, 31])

In [14]:
torch.ones([1, 1, 31, 721, 1440]).flatten()[:5]

tensor([1., 1., 1., 1., 1.])

In [44]:
from modulus.models_new.graphcast import GraphCast

model = GraphCast(
    sample_dimensions=sample_image.shape,
    grid_input_dimension=31,
    grid_output_dimension=21
)

model(sample_image)

IndentationError: expected an indented block after function definition on line 99 (mesh_graph_mlp.py, line 100)